Ref:

1. https://stackoverflow.com/questions/55240940/error-while-installing-spark-on-google-colab
2. https://github.com/WhitewaterFoundry/Pengwin/issues/448
3. https://stackoverflow.com/questions/55595263/how-to-fix-no-filesystem-for-scheme-gs-in-pyspark

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
#!sudo apt-get install default-jre default-jdk

# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
#os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

openjdk version "11.0.18" 2023-01-17
OpenJDK Runtime Environment (build 11.0.18+10-post-Debian-1deb10u1)
OpenJDK 64-Bit Server VM (build 11.0.18+10-post-Debian-1deb10u1, mixed mode, sharing)


In [ ]:
'''
# start spark session
#import sparknlp
#import findspark
#findspark.init()
#spark = sparknlp.start()

print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)
! cd ~/.ivy2/cache/com.johnsnowlabs.nlp/spark-nlp_2.12/jars && ls -lt'''

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Create spark_session
#spark = SparkSession.builder.getOrCreate()

spark = SparkSession \
        .builder \
        .config("spark.jars", "/path/to/gcs-connector-hadoop2-latest.jar") \
        .getOrCreate()

sc = spark.sparkContext
spark.version

spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

23/05/09 06:25:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/09 06:25:53 WARN DependencyUtils: Local jar /path/to/gcs-connector-hadoop2-latest.jar does not exist, skipping.
23/05/09 06:25:53 INFO SparkContext: Running Spark version 3.3.0
23/05/09 06:25:53 INFO ResourceUtils: ==============================================================
23/05/09 06:25:53 INFO ResourceUtils: No custom resources configured for spark.driver.
23/05/09 06:25:53 INFO ResourceUtils: ==============================================================
23/05/09 06:25:53 INFO SparkContext: Submitted application: pyspark-shell
23/05/09 06:25:53 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amount: 1024, script: , vendor: , offHeap -> name: offHeap, amount: 0, script: , vendor: ), task resources: Map(cpus -> name: cpus, amoun

In [7]:
from google.cloud import storage

# Reading data from open bucket
#dataPath = 'gs://nlp_final_spark/check_points/checkpoint.parquet'
dataPath = './data/checkpoint.parquet'

In [8]:
!ls

data				       nlp_final_1_02_PySpark.ipynb
nlp_final_1_01_data_preparation.ipynb  spark-3.3.2-bin-hadoop3


In [ ]:
# read data
news_df = spark.read.parquet(dataPath)

In [ ]:
# List all files in given COS directory
def list_blobs(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    for blob in blobs:
        print(blob.name + '\t' + str(blob.size))

# List all files in given COS directory
def list_blobs_pd(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    blob_name = []
    blob_size = []
    
    for blob in blobs:
        blob_name.append(blob.name)
        blob_size.append(blob.size)

    blobs_df = pd.DataFrame(list(zip(blob_name, blob_size)), columns=['Name','Size'])

    blobs_df = blobs_df.style.format({"Size": "{:,.0f}"}) 
    
    return blobs_df        

# Delete folder from COS bucket
def delete_folder(bucket_name, folder_name):
    gcs_client = storage.Client()
    bucket = gcs_client.bucket(bucket_name)
    blobs = list(bucket.list_blobs(prefix=folder_name))

    for blob in blobs:
        blob.delete()

In [ ]:
# Saving results into individual bucket, students must update to their own bucket `msca-bdp-students-bucket` and use `CNET ID` as a folder prefix
bucket_write = 'nlp_final_spark'
folder_write = 'check_points'

In [ ]:
'''#%%time
data_joined.write.format('parquet').\
mode('overwrite').\
save('gs://' + bucket_write + '/' + folder_write)'''